In [1]:
import pandas as pd 
import chardet 
import numpy as np
from sklearn.model_selection import train_test_split
import operator
from sklearn.metrics import confusion_matrix 
from sklearn.cross_validation import train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

C:\Users\swami\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
with open("spam.csv", 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large

#print(result['encoding'])
df = pd.read_csv('spam.csv', encoding=result['encoding'])
df=df.loc[:,'v1':'v2']
df=df.rename(index=str, columns={"v1": "status", "v2": "message"})

# Naive Model
I split the data set into test and train. Using train set I find the most freqently occurring words in spam sms's.
Then I modify the dataframe and give in new features to the model.
I use the test data to calculate accuracies and other statistics.
The naive model classifies an sms as spam when one of the frequently occurring spam words is found in the sms.

In [3]:
X = df['message']
y = df['status']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)
d={}
def tfidf (a,s):#a is the spam ham list 
    for i in range(len(a)):
        if s[i]=="spam":
            for x in list(a[i].split(' ')):
                d[x]=[0,0]
                
    for i in range(len(a)):
        if s[i]=="spam":
            for x in list(a[i].split(' ')):
                d[x][0]=d[x][0]+1
            for x in set(list(a[i].split(' '))):
                d[x][1]=d[x][1]+1
        
tfidf(X_train,y_train)
k=len(X_train)
for x in d:
    d[x]=d[x][0]*np.log((k/d[x][1])) 
d=sorted(d.items(), key=operator.itemgetter(1))
# frequently occurring spamwords picked manually 
# FREE  Free won NOW! chance CALL Send Please Get service STOP cash prize
def change(x):
    if(x == 'ham'):
        return 0
    else:
        return 1
def frq(x):
    return(int('STOP' in x )+int('prize' in x )+int('cash' in x )+int('service' in x )+int('Get' in x )+
           int('Please' in x )+int('Send' in x )+int('CALL' in x )+int('chance' in x )+int('NOW!' in x )+
           int('won' in x )+int('Free' in x )+int('FREE' in x ))

df['status'] = df['status'].map(lambda x : change(x))
df['len'] = df['message'].map(lambda x: len(x))
df['FREE'] = df['message'].map(lambda x: int('FREE' in x ))
df['Free'] = df['message'].map(lambda x: int('Free' in x ))
df['won'] = df['message'].map(lambda x: int('won' in x ))
df['NOW!'] = df['message'].map(lambda x: int('NOW!' in x ))
df['chance'] = df['message'].map(lambda x: int('chance' in x ))
df['CALL'] = df['message'].map(lambda x: int('CALL' in x ))
df['Send'] = df['message'].map(lambda x: int('Send' in x ))
df['Please'] = df['message'].map(lambda x: int('Please' in x ))
df['Get'] = df['message'].map(lambda x: int('Get' in x ))
df['service'] = df['message'].map(lambda x: int('service' in x ))
df['cash'] = df['message'].map(lambda x: int('cash' in x ))
df['prize'] = df['message'].map(lambda x: int('prize' in x ))
df['STOP'] = df['message'].map(lambda x: int('STOP' in x ))
df['freq'] = df['message'].map(lambda x : frq(x))
gg=[]
true_positive=0
true_negative=0
false_positive=0
false_negative=0
for i in range(len(X_test)):
    gg.append([X_test[i],y_test[i],frq(X_test[i])])
    if y_test[i]=="spam":
        if frq(X_test[i])==0:
            false_positive=false_positive+1
        else:
            true_positive=true_positive+1
    else:
        if frq(X_test[i])==0:
            true_negative=false_negative+1
        else:
            true_negative=true_negative+1

In [4]:
precision=true_positive/(true_positive+false_positive)
recall=true_positive/(true_positive+false_negative)
print("accuracy",(true_negative+true_positive)/(true_negative+true_positive+false_negative+false_positive))
print("precision",precision)
print("recall",recall)
print("f1",2*precision*recall/(precision+recall))

accuracy 0.49740932642487046
precision 0.4947916666666667
recall 1.0
f1 0.662020905923345


# Decision Trees
Decision trees are built by choosing the right attribute at each node of a tree. The attribute is chosen such that the information gain is high among all attributes. Entropy and Gini Index are the most commonly used function for a decision tree classifier. I have used Gini Index for this classifier.


In [5]:
X_dt = df[['len','FREE','Free','won','NOW!','chance','cash','prize','freq']]
y_dt = df['status']
X_train_dt, X_test_dt, y_train_dt, y_test_dt = train_test_split(X_dt, y_dt, random_state=10)

      
# Function to perform training with giniIndex. 
def train_using_gini(X_train, X_test, y_train): 
    # Creating the classifier object 
    clf_gini = DecisionTreeClassifier(criterion = "gini", 
            random_state = 100,max_depth=3, min_samples_leaf=5) 
    # Performing training 
    clf_gini.fit(X_train, y_train) 
    return clf_gini 
# Function to make predictions 
def prediction(X_test, clf_object): 
    # Predicton on test with giniIndex 
    y_pred = clf_object.predict(X_test) 
    print("Predicted values:") 
    return y_pred 
# Function to calculate accuracy 
def cal_accuracy(y_test, y_pred): 
    print("Confusion Matrix: ", 
        confusion_matrix(y_test, y_pred)) 
    print ("Accuracy : ", 
    accuracy_score(y_test,y_pred)*100) 
    print("Report : ", 
    classification_report(y_test, y_pred)) 
clf_gini = train_using_gini(X_train_dt, X_test_dt, y_train_dt)      
# Prediction using gini 
y_pred_gini = prediction(X_test_dt, clf_gini) 
cal_accuracy(y_test_dt, y_pred_gini) 

Predicted values:
Confusion Matrix:  [[1183   18]
 [ 120   72]]
Accuracy :  90.09332376166547
Report :               precision    recall  f1-score   support

          0       0.91      0.99      0.94      1201
          1       0.80      0.38      0.51       192

avg / total       0.89      0.90      0.89      1393



# Random Forest
Random forest builds multiple decision trees and merges them together to get a more accurate and stable prediction.
One big advantage of random forest is, that it can be used for both classification and regression problems. Random Forest has nearly the same hyperparameters as a decision tree or a bagging classifier. Random Forest adds additional randomness to the model, while growing the trees. Instead of searching for the most important feature while splitting a node, it searches for the best feature among a random subset of features. This results in a wide diversity that generally results in a better model.

In [6]:
X_rf = df[['len','FREE','Free','won','NOW!','chance','cash','prize','freq']]
y_rf = df['status']
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_rf, y_rf, random_state=10)
rf = RandomForestClassifier(n_estimators=20, max_depth=5, criterion= "entropy")
train_features = np.array(X_train_rf) 
train_labels = np.array(y_train_rf)
rf.fit(train_features, train_labels) 
test_features = np.array(X_test_rf) 
test_labels = np.array(y_test_rf) 
test_predict = rf.predict(test_features)


print("accuracy",accuracy_score(test_predict, test_labels))
print("f1",f1_score(test_predict, test_labels, average = 'binary'))
print("confusion_matrix",confusion_matrix(test_labels, test_predict))

accuracy 0.9109834888729361
f1 0.581081081081081
confusion_matrix [[1183   18]
 [ 106   86]]
